# Define Functions and Run Initial Query

In [1]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 10

def read_query_file(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

In [5]:
dfp = run_query(read_query_file('SQL\SamplingTest\KeyMetrics_ForGroupSelection-QuartileTest.SQL'))
display(dfp.head())

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_20864\1332871533.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Employee,TenureGroup,Location,Supervisor,Date,HoursWorked,Sessions,Total_Resolution_Time,Total_Accepted,Total_Eligible,Helix_Searches,Helix_Sessions,CXP_Prob_Sum,CXP_Prob_Count
0,493122,180+,tcpn,joshua tesoro,2023-07-07,7.57,10.00,11254,1.00,5.00,4,2,3.10,6
1,493122,180+,tcpn,joshua tesoro,2023-07-24,7.94,8.00,14602,1.00,5.00,6,3,6.15,12
2,493122,180+,tcpn,joshua tesoro,2023-07-23,8.00,7.00,10853,1.00,8.00,10,5,6.78,13
3,493122,180+,tcpn,joshua tesoro,2023-07-21,5.65,5.00,6348,0.00,3.00,0,0,3.09,6
4,493122,180+,tcpn,joshua tesoro,2023-07-16,7.96,8.00,14433,2.00,6.00,0,0,5.19,10


# Select Sample Size by Tenure Group
Using the current number of experts find the number of samples needed in each group to be representative of the general population at Orlando

In [7]:
dfp['TestGroup'] = (dfp['Employee'].isin((573573,576911,572909,577073,575731,573585,573192,573276,572815,572247,579162,577246,573190,
                                        576565,575996,581139,581145,580911,580895,581015,581275,552121,569375,547655,426097,554487
))).astype(int)
df=dfp[dfp['Sessions']!=0].copy()
df = df[~df['Sessions'].isna()]
df = pd.pivot_table(df,index=['Employee','Supervisor','Location','TenureGroup'],
                     values=['Total_Accepted','Total_Eligible','CXP_Prob_Sum','CXP_Prob_Count','Sessions','HoursWorked','Total_Resolution_Time','Helix_Searches','Helix_Sessions'])

display(100*dfp['TenureGroup'].value_counts()/dfp['TenureGroup'].value_counts().sum())
df['SP100'] = df['Total_Accepted']/df['Total_Eligible']
df['CXP_Score']=df['CXP_Prob_Sum']/df['CXP_Prob_Count']
df['Sessions Per Hour'] = df['Sessions']/df['HoursWorked']
df['CRT']= df['Total_Resolution_Time']/df['Sessions']
df['Searches Per Session']= df['Helix_Searches']/df['Sessions']
df['%Sessions With Search']= df['Helix_Sessions']/df['Sessions']
df = df[['Sessions Per Hour','CRT','CXP_Score','%Sessions With Search','Searches Per Session','Total_Accepted','SP100']]
df = df.dropna()
display(df,df.describe())
df.to_excel('Quartile Data.xlsx')
df.describe().to_excel('Quartile Describe.xlsx')

180+      74.41
91-120     8.49
121-180    8.39
61-90      4.88
31-60      3.18
0-30       0.65
Name: TenureGroup, dtype: float64

,,,,Sessions Per Hour,CRT,CXP_Score,%Sessions With Search,Searches Per Session,Total_Accepted,SP100
Employee,Supervisor,Location,TenureGroup,,,,,,,
131514,ali valdespino,waha,180+,2.29,978.11,0.52,0.11,0.40,0.62,0.07
147378,dekay pope,waha,180+,1.64,901.71,0.51,0.14,0.19,0.78,0.13
171913,dekay pope,waha,180+,1.24,"1,386.49",0.52,0.17,0.62,0.42,0.11
176427,stephanie smith,waha,180+,2.10,888.13,0.49,0.02,0.05,0.00,0.00
177743,stephanie williams,waha,180+,0.31,"1,881.67",0.53,0.33,0.33,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
593932,maria elena teresa concepcion,pclk,0-30,0.61,"1,416.05",0.53,0.50,1.30,0.00,0.00
593934,maria elena teresa concepcion,pclk,0-30,0.59,"1,695.11",0.51,0.79,2.05,0.00,0.00
593940,maria elena teresa concepcion,pclk,0-30,0.53,"2,079.76",0.51,0.53,2.29,0.00,0.00


,Sessions Per Hour,CRT,CXP_Score,%Sessions With Search,Searches Per Session,Total_Accepted,SP100
count,"2,452.00","2,452.00","2,452.00","2,452.00","2,452.00","2,452.00","2,452.00"
mean,1.49,"1,321.93",0.52,0.12,0.26,0.70,0.09
std,0.40,279.53,0.00,0.14,0.36,0.45,0.05
mad (mean),0.30,213.15,0.00,0.09,0.23,0.35,0.04
mad (median),0.23,166.03,0.00,0.07,0.19,0.30,0.03
min,0.15,145.00,0.49,0.00,0.00,0.00,0.00
25%,1.27,"1,145.86",0.51,0.04,0.06,0.37,0.05
50%,1.51,"1,295.08",0.52,0.08,0.13,0.65,0.08
75%,1.73,"1,481.29",0.52,0.15,0.32,0.96,0.12
max,3.55,"2,982.38",0.54,1.40,4.40,3.25,0.34
